> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `Lib-INVENT`: Reinforcement Learning - ROCS + reaction filter
The purpose of this notebook is to illustrate the assembly of a configuration input file containing a ROCS input.

ROCS is a licensed virtual screening software based on similarity between input compounds and a specified reference (or target) molecule. For more information on ROCS, please refer to the OpenEye website: https://www.eyesopen.com/rocs

*Note that in order to use ROCS, an OpenEye license is needed.*

## Setting up the environment for registering the license

If you have obtained the license you need to setup a few things to actually register the license.

1. First, go to the `$CONDA_PREFIX` directory of your Conda environment for Reinvent, _not_ the one for ReinventCommunity.

2. Run the following command:

```
$ mkdir -p etc/conda/{de,}activate.d
```

3. Create an `etc/conda/activate.d/env_vars.sh` file with the following contents:

```sh
#!/bin/sh
export OE_LICENSE='/path/to/your/openeye/license' # change this to the appropriate path
```

4. Create an `etc/conda/deactivate.d/env_vars.sh` file with the following contents:

```sh
#!/bin/sh
unset OE_LICENSE
```

5. Reactivate your Conda environment, and if everything went well, `echo $OE_LICENSE` should output the path to your OpenEye license that you specified in the `etc/conda/activate.d/env_vars.sh` file.

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent.v3.2")
output_dir = os.path.expanduser("~/Desktop/REINVENT_RL_ROCS_RF_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## Setting up the configuration
The configuration is set up analogously to the previous tutorials. The difference arrises in the scoring function composition where a QSAR predictive property is not replaced with the ROCS model.

The Amide-coupling/Buchwald reaction filters that were previously discussed in `Lib-INVENT_RL2_QSAR_RF` are imposed.

### 1. Run Type Block

In [2]:
# initialize the dictionary
configuration = {
    "version": 3,
    "model_type": "lib_invent",
    "run_type": "reinforcement_learning"
}

### 2. Logging Block

In [3]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "",                          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_path": os.path.join(output_dir, "progress.log"), # load this folder in tensorboard
    "result_folder": os.path.join(output_dir, "results"), # output directory for results
    "job_name": "Reinforcement learning ROCS + RF demo",    # set an arbitrary job name for identification
    "job_id": "n/a"                        # only relevant if "recipient" is set to "remote"
}

### 3. Parameters Block

In [4]:
# start assembling parameters block
configuration["parameters"] = {
    "actor": os.path.join(ipynb_path, "models/library_design.prior"),
    "critic": os.path.join(ipynb_path, "models/library_design.prior"),
    "scaffolds": ["[*:0]N1CCN(CC1)CCCCN[*:1]"],
    "n_steps": 100,
    "learning_rate": 0.0001,
    "batch_size": 128,
    "randomize_scaffolds": False, # important since a RF is to be imposed

    # configure learning strategy
    "learning_strategy": {
        "name": "dap", 
        "parameters":  {
          "sigma": 120
        }
    }
}

#### Scoring strategy

In [5]:
# configure scoring strategy
scoring_strategy = {
    "name": "lib_invent", # Do not change


    # configure diversity filter
    "diversity_filter": {
        "name": "NoFilterWithPenalty",     # Use a DF; the alternative option is "NoFilter"
    },


    # configure reaction filter
    "reaction_filter":  {
        "type":"selective",        
        "reactions":[
            ["[#6;!$(C(C=*)(C=*));!$([#6]~[O,N,S]);$([#6]~[#6]):1][C:2](=[O:3])[N;D2;$(N(C=[O,S]));!$(N~[O,P,S,N]):4][#6;!$(C=*);!$([#6](~[O,N,S])N);$([#6]~[#6]):5]>>[#6:1][C:2](=[O:3])[*].[*][N:4][#6:5]"],
            ["[c;$(c1:[c,n]:[c,n]:[c,n]:[c,n]:[c,n]:1):1]-!@[N;$(NC)&!$(N=*)&!$([N-])&!$(N#*)&!$([ND1])&!$(N[O])&!$(N[C,S]=[S,O,N]),H2&$(Nc1:[c,n]:[c,n]:[c,n]:[c,n]:[c,n]:1):2]>>[*][c;$(c1:[c,n]:[c,n]:[c,n]:[c,n]:[c,n]:1):1].[*][N:2]"]
        ]
    }
}

#####  Define the scoring function
This is the important point of difference between this tutorial and `Lib-INVENT_RL2_QSAR_RF`. A ROCS similarity measure is imposed to guide the agent to propose compounds structurally resembling a known haloperidol ligand.

In [6]:
scoring_function = {
    "name": "custom_sum",              
    "parallel": False,                     
    "parameters": [
    {
        "component_type": "parallel_rocs_similarity",
        "name": "RefTversky ROCS sim",
        "specific_parameters": {
            "shape_weight": 0.5,
            "color_weight": 0.5,
            "custom_cff": os.path.join(ipynb_path, "data/implicit_MD_mod_generic_hydrophobe.cff"),
            "enumerate_stereo": True,
            "input_type": "shape_query",
            "max_num_cpus": 8,
            "rocs_input": os.path.join(ipynb_path, "data/haloperidol_3_feats.sq"),
            "similarity_measure": "RefTversky",
            "transformation": {
                "transformation_type": "no_transformation"
            }
        },
        "weight": 1
    },
    {
        "component_type": "custom_alerts",
        "name": "Custom alerts",              
        "weight": 1,                                            
        "specific_parameters": {
            "smiles": [                           
                "[*;r8]",
                "[*;r9]",
                "[*;r10]",
                "[*;r11]",
                "[*;r12]",
                "[*;r13]",
                "[*;r14]",
                "[*;r15]",
                "[*;r16]",
                "[*;r17]",
                "[#8][#8]",
                "[#6;+]",
                "[#16][#16]",
                "[#7;!n][S;!$(S(=O)=O)]",
                "[#7;!n][#7;!n]",
                "C#C",
                "C(=[O,S])[O,S]",
                "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
                "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
                "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
                "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
                "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
                "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
            ]
        }          
    }]
}

scoring_strategy["scoring_function"] = scoring_function

In [7]:
# Update the paramters block with the scoring strategy
configuration["parameters"]["scoring_strategy"] = scoring_strategy

## Write out the configuration

In [8]:
configuration_JSON_path = os.path.join(output_dir, "RL_ROCS_RF_input.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

# Run
Execute in jupyter notebook

In [9]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!{reinvent_env}/bin/python {reinvent_dir}/input.py {configuration_JSON_path}

In [10]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

Execute in command line
```
$ conda activate reinvent.v3.2
$ python <reinvent_dir>/input.py <configuration_JSON_path>
```

## Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`, as described in the `Lib-INVENT_RL1_QSAR` tutorial.

```
# go to the root folder of the output
$ cd <output_dir>

# activate the environment
$ conda activate reinvent.v3.2

# start tensorboard
$ tensorboard --logdir "progress.log"
```